In [1]:
import re

import pandas as pd
import plotly.express as px

from Project.Database import Db

In [11]:
def create_gantt_vis(year):
    NZERTF, NZERTF_meta = Db.load_data(hourly=False, meta=True, year=year)

    #Extract appliances from the meta data
    appliance_condition = (lambda self: (
            (self["Parameter"] == "Status_OnOff") &
            (self["Subsystem"] == "Loads") &
            (~self.index.str.contains("SensHeat"))
    ))
    appliance_location = pd.DataFrame(NZERTF_meta.loc[appliance_condition, "Measurement_Floor"].sort_values(ascending=True))
    appliance_order = NZERTF_meta.loc[appliance_condition, ["Measurement_Floor", "Measurement_Location"]]
    appliance_order["Name"] = [
        " ".join(re.findall('[A-Z][^A-Z]*', "".join(''.join(app.split("PlugLoad")).split("Appliance")).split("Load_Status")[-1])) for app in
        appliance_order.index]
    appliance_order = appliance_order.sort_values(by=["Measurement_Floor", "Measurement_Location", "Name"])["Name"]

    #Combine minute data with the extracted appliances into a dataframe
    minute_appliances_status = NZERTF[["Timestamp"] + appliance_location.index.tolist()].copy()

    appliance_job_list = []

    for appliance, appliance_row in appliance_location.iterrows():
        name = appliance_order[appliance]
        appliance_switch = minute_appliances_status[(minute_appliances_status[appliance] != minute_appliances_status[appliance].shift(1))][["Timestamp", appliance]][1:]
        appliance_switch.reset_index(inplace=True, drop=True)
        for index, row in appliance_switch.iterrows():
            if row[appliance]:
                try:
                    appliance_job_list.append({"appliance": name, "start": row["Timestamp"],
                                               "end": appliance_switch.loc[index + 1, "Timestamp"],
                                               "location": {'1stFloor': "First floor", '2ndFloor': 'Second floor'}[appliance_row["Measurement_Floor"]]})
                except:
                    continue

    # px.timeline(appliance_job_list, x_start="start", x_end="end", y="appliance", color="location", category_orders={"appliance": appliance_order.tolist() + ["PrntA", "PrntB", "ChildA", "ChildB"]}).write_html(
    #     Db.get_save_file_directory(f"Gantt/Appliance_status_gantt_year_{year}_group_floor.html"))

    residens_condition = (lambda self: (
            (self["Subsystem"] == "Loads") &
            (self["Parameter"] == "Status_OnOff") &
            (self.index.str.contains("Sens"))
    ))

    residens_location = NZERTF_meta.loc[residens_condition].index.tolist()
    residens_location.sort()

    #Combine minute data with the extracted residens into a dataframe
    residens_location_status = NZERTF[["Timestamp"] + residens_location].copy()
    person_status_list = []

    for col in residens_location:
        name = "".join(re.findall("[PC][a-z]+[AB]", col))
        person_location = residens_location_status[lambda self: (self[col] != self[col].shift(1))][
                              ["Timestamp", col]][1:]

        df_indices = person_location.index.tolist()
        for index, df_index in enumerate(df_indices):
            if person_location.loc[df_index, col]:
                try:
                    person_status_list.append({"appliance": name, "start": person_location.loc[df_index, "Timestamp"],
                                               "end": person_location.loc[df_indices[index + 1], "Timestamp"],
                                               "location": {"DOWN": "First floor", "UP": "Second floor"}[
                                                   "".join(col.split(name)[-1])]})
                except:
                    continue

    fig = px.timeline(person_status_list + appliance_job_list, x_start="start", x_end="end", y="appliance", color="location", category_orders={"appliance": appliance_order.tolist() + ["PrntA", "PrntB", "ChildA", "ChildB"]})
    fig.update_layout(template='plotly')
    fig.write_html(Db.get_save_file_directory(f"Gantt/Person_appliance_status_gantt_year_{year}_group_floor.html"))

In [12]:
for year in [1, 2]:
    create_gantt_vis(year)